In [2]:
import os
os.path.abspath("")

'/home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting/notebook'

## Load data

In [3]:
import pandas as pd
REPORT_PATH = "/home/yuxiangliao/PhD/data/mimic_cxr_reports_core.json"
df = pd.read_json(REPORT_PATH,orient="records",lines=True)
print(df)

id_list = df.loc[:,'sid'].to_list()
findings_list = df.loc[:,'findings'].to_list()
impression_list = df.loc[:,'impression'].to_list()
pfi_list = df.loc[:,'provisional_findings_impression'].to_list()
fai_list = df.loc[:,'findings_and_impression'].to_list()

DATA_SIZE = len(id_list)

              pid        sid  \
0       p10000032  s50414267   
1       p10000032  s53189527   
2       p10000032  s53911762   
3       p10000032  s56699142   
4       p10000764  s57375967   
...           ...        ...   
227830  p19999442  s58708861   
227831  p19999733  s57132437   
227832  p19999987  s55368167   
227833  p19999987  s58621812   
227834  p19999987  s58971208   

                                                 findings  \
0       There is no focal consolidation, pleural effus...   
1       The cardiac, mediastinal and hilar contours ar...   
2       Single frontal view of the chest provided. \n ...   
3       The lungs are clear of focal consolidation, pl...   
4       PA and lateral views of the chest provided.   ...   
...                                                   ...   
227830  ET tube ends 4.7 cm above the carina.  NG tube...   
227831  The lungs are clear, and the cardiomediastinal...   
227832  There has been interval extubation and improve...   
22783

# Check CoreNLP

In [4]:
!ls $CORENLP_HOME

build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.2.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.0.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.0-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.0-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.0-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

# CoreNLP

# CoreNLPClient

In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

config_path = "../src/config/"
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=config_path, version_base=None)
config = compose(config_name="nlp_ensemble", overrides=["nlp_ensemble@_global_=i2b2"])

print(OmegaConf.to_yaml(config))

machine:
  work_dir: /home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting
  fast_coref_dir: /home/yuxiangliao/PhD/workspace/git_clone_repos/fast-coref/
work_dir: ${machine.work_dir}
src_dir: ${work_dir}/src
output_dir: ${work_dir}/output
base_output_dir: ${work_dir}/output
mimic_cxr_output_dir: ${base_output_dir}/mimic_cxr
i2b2_output_dir: ${base_output_dir}/i2b2
log_dir: ${work_dir}/logs/${hydra.job.config_name}
logging_level: INFO
ignore_source_path: ${work_dir}/config/ignore/common_ignore
fast_coref_dir: ${machine.fast_coref_dir}
coref_scorer_dir: ${machine.fast_coref_dir}/coref_resources/reference-coreference-scorers
name_style:
  corenlp:
    prefix: '[co]'
    statistical_coref_prefix: '[ml]'
    nerual_coref_prefix: '[nn]'
    deterministic_coref_prefix: '[rb]'
    column_name:
      align_to_spacy: ${name_style.corenlp.prefix}align_to_spacy
      token: ${name_style.corenlp.prefix}token
      token_offset: ${name_style.corenlp.prefix}token_offset
      sentence

In [2]:
import stanza
from stanza.server import CoreNLPClient
import time
import json
COREF_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, depparse, coref',
    "coref.algorithm": "statistical",
    'outputFormat': 'json',
}
DCOREF_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, parse, dcoref',
    'outputFormat': 'json',
}
NEURAL_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, parse, coref',
    'coref.algorithm': 'neural ',
    'outputFormat': 'json',
}
client = CoreNLPClient(memory='8G', threads=14, endpoint='http://localhost:8802', be_quiet=False,output_format="JSON", properties=DCOREF_PROPS)
client.start()
client.ensure_alive()

2022-09-17 13:19:31 INFO: Writing properties to tmp file: corenlp_server-c597c0c87cf64f81.props
2022-09-17 13:19:31 INFO: Starting server with command: java -Xmx8G -cp /home/yuxiangliao/PhD/Stanford_CoreNLP/stanford-corenlp-4.5.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 8802 -timeout 60000 -threads 14 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-c597c0c87cf64f81.props -preload -outputFormat JSON
[main] INFO CoreNLP - --- StanfordCoreNLPServer#main() called ---
[main] INFO CoreNLP -     Build: x86_64-conda-linux-gnu
[main] INFO CoreNLP - Server default properties:
			(Note: unspecified annotator properties are English defaults)
			annotators = tokenize, ssplit, pos, lemma, ner, parse, dcoref
			inputFormat = text
			outputFormat = JSON
			prettyPrint = false
			threads = 14
[main] INFO CoreNLP - Threads: 14
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding

In [3]:
import sys
sys.path.append("../src/")
from nlp_ensemble.nlp_processor.corenlp_process import CorenlpUrlProcess, formatCorenlpDocument
from common_utils.nlp_utils import resolveTokenIndices_byPosition
from common_utils.nlp_utils import align_byIndex_individually_nestedgruop, align_byIndex_individually_withData_dictInList, align_coref_groups_in_conll_format, align_byIndex_individually_withData_noOverlap
from common_utils.nlp_utils import align, getTokenOffset
from nlp_ensemble.nlp_processor.spacy_process import SpacyProcess, init_spacy
from common_utils.ensemble_utils import load_i2b2

input_path = os.path.join("/home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting/output/i2b2/i2b2_all.jsonlines")

json_name_cfg = config.name_style.i2b2.json
id_list, section_list = load_i2b2(input_path, json_name_cfg)
print(len(id_list))

424


In [4]:
import requests
import pandas as pd

sid = 'clinical-67'

t = section_list[0][1][id_list.index(sid)]


request_url = 'http://0.0.0.0:8802/'
corenlp_out_jsonStr = requests.post(request_url, data=t.encode()).text



[pool-1-thread-3] INFO CoreNLP - [/127.0.0.1:44942] API call w/annotators tokenize,pos,lemma,ner,parse,dcoref
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator dcoref


869436718 SC 64806295 6613471 10/1/2003 12:00:00 AM post partum DIS Admission Date : 10/01/2003 Report Status : Discharge Date : 10/04/2003 ****** DISCHARGE ORDERS ****** KULRINNE , RYYELEGE N 386-85-38-0 X32 Room : JQL5-545 Service : OBS DISCHARGE PATIENT ON : 10/04/03 AT 10:00 AM CONTINGENT UPON Not Applicable WILL D / C ORDER BE USED AS THE D / C SUMMARY : YES Attending : KOTEJOASTHOREUPHKAYS , GAI , M.D. DISPOSITION : Home DISCHARGE MEDICATIONS : TYLENOL ( ACETAMINOPHEN ) 650 MG PO Q4H PRN PAIN Alert overridden : Override added on 10/01/03 by KOTEJOASTHOREUPHKAYS , GAI , M.D. POTENTIALLY SERIOUS INTERACTION : ACETAMINOPHEN and ACETAMINOPHEN Reason for override : ok COLACE ( DOCUSATE SODIUM ) 100 MG PO BID IBUPROFEN 600 MG PO Q4H PRN PAIN Food / Drug Interaction Instruction Take with food MILK OF MAGNESIA ( MAGNESIUM HYDROXIDE ) 30 MILLILITERS PO BID PRN Constipation PERCOCET 1-2 TAB PO Q4H PRN PAIN Override Notice : Override added on 10/01/03 by KOTEJOASTHOREUPHKAYS , GAI , M.D. on

[pool-2-thread-1] WARN edu.stanford.nlp.pipeline.ParserAnnotator - Parsing of sentence failed, possibly because of out of memory.  Will ignore and continue: 869436718 SC 64806295 6613471 10/1/2003 12:00:00 AM post partum DIS Admission Date : 10/01/2003 Report Status : Discharge Date : 10/04/2003 ****** DISCHARGE ORDERS ****** KULRINNE , RYYELEGE N 386 -85-38-0 X32 Room : JQL5 - 545 Service : OBS DISCHARGE PATIENT ON : 10/04/03 AT 10:00 AM CONTINGENT UPON Not Applicable WILL D / C ORDER BE USED AS THE D / C SUMMARY : YES Attending : KOTEJOASTHOREUPHKAYS , GAI , M.D. DISPOSITION : Home DISCHARGE MEDICATIONS : TYLENOL ( ACETAMINOPHEN ) 650 MG PO Q4H PRN PAIN Alert overridden : Override added on 10/01/03 by KOTEJOASTHOREUPHKAYS , GAI , M.D. POTENTIALLY SERIOUS INTERACTION : ACETAMINOPHEN and ACETAMINOPHEN Reason for override : ok COLACE ( DOCUSATE SODIUM ) 100 MG PO BID IBUPROFEN 600 MG PO Q4H PRN PAIN Food / Drug Interaction Instruction Take with food MILK OF MAGNESIA ( MAGNESIUM HYDROXID

In [5]:
print(corenlp_out_jsonStr)

{
  "sentences": [
    {
      "index": 0,
      "parse": "(X (CD 869436718) (NN SC) (CD 64806295) (CD 6613471) (CD 10/1/2003) (CD 12:00:00) (NN AM) (NN post) (NN partum) (NN DIS) (NN Admission) (NN Date) (: :) (CD 10/01/2003) (NNP Report) (NNP Status) (: :) (NNP Discharge) (NN Date) (: :) (CD 10/04/2003) (NFP ******) (NNP DISCHARGE) (NNS ORDERS) (NFP ******) (NN KULRINNE) (, ,) (NN RYYELEGE) (NN N) (CD 386) (NN -85-38-0) (NN X32) (NN Room) (: :) (NN JQL5) (HYPH -) (CD 545) (NN Service) (: :) (NNP OBS) (NNP DISCHARGE) (NN PATIENT) (IN ON) (: :) (CD 10/04/03) (NN AT) (CD 10:00) (NN AM) (NNP CONTINGENT) (NNP UPON) (RB Not) (NNP Applicable) (MD WILL) (NNP D) (SYM /) (NNP C) (NN ORDER) (VB BE) (VBN USED) (IN AS) (DT THE) (NN D) (HYPH /) (NN C) (NN SUMMARY) (: :) (UH YES) (VBG Attending) (: :) (NNP KOTEJOASTHOREUPHKAYS) (, ,) (NNP GAI) (, ,) (NNP M.D.) (NNP DISPOSITION) (: :) (NNP Home) (NNP DISCHARGE) (NNS MEDICATIONS) (: :) (NN TYLENOL) (-LRB- -LRB-) (NN ACETAMINOPHEN) (-RRB- -RRB-) (CD 6

In [7]:
client.stop()

# Stanza

Biomedical models

In [ ]:
# import stanza
# stanza.download('en', package='mimic')
# stanza.download('en', package='radiology')

Stanza processor: https://stanfordnlp.github.io/stanza/pipeline.html#processors

In [ ]:
# from stanza.pipeline.core import DownloadMethod
# import stanza

# processor_dict = {
#     'tokenize': 'mimic', 
#     'pos': 'mimic', 
#     'lemma': 'mimic',
#     'depparse': 'mimic',
#     # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
#     'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
#     'ner': 'radiology',
# }
# nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
#                       download_method=DownloadMethod.REUSE_RESOURCES,
#                       verbose=False) # logging_level='WARN'
# doc = nlp(findings_list[8690])
# # print out dependency tree
# print(doc)

Processing Multiple Documents

28s -> 100docs

In [10]:
import stanza
from stanza.pipeline.core import DownloadMethod

processor_dict = {
    'tokenize': 'mimic', 
    'pos': 'mimic', 
    'lemma': 'mimic',
    'depparse': 'mimic',
    # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
    'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
    'ner': 'radiology',
}

def set_sid(self, value):
    self._sid = value
def get_sid(self):
    return self._sid

nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
                      download_method=DownloadMethod.REUSE_RESOURCES,
                      verbose=False) # logging_level='WARN'
documents = findings_list[8690:8691] # Documents that we are going to process
# stanza.Document.add_property("sid",getter=get_sid, setter=set_sid)


NER tag
- B，Begin
- I，Intermediate
- E，End
- S，Single
- O，Other

In [11]:
in_docs = []
for id,d in enumerate(documents):
    stanzaDoc = stanza.Document([], text=d)
    stanzaDoc._sid = id_list[id]
    in_docs.append(stanzaDoc)
# in_docs = [stanza.Document([], text=d) for d in documents] # Wrap each document with a stanza.Document object
out_docs = nlp(in_docs) # Call the neural pipeline on this list of documents
# print(out_docs[0]) # The output is also a list of stanza.Document objects, each output corresponding to an input Document object
for doc in out_docs:
    print(f"sid:{doc._sid}")
    print(doc)

sid:s50414267
[
  [
    {
      "id": 1,
      "text": "Compared",
      "lemma": "compare",
      "upos": "VERB",
      "xpos": "VBN",
      "head": 5,
      "deprel": "case",
      "start_char": 0,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "with",
      "lemma": "with",
      "upos": "ADP",
      "xpos": "IN",
      "head": 5,
      "deprel": "case",
      "start_char": 9,
      "end_char": 13,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "the",
      "lemma": "the",
      "upos": "DET",
      "xpos": "DT",
      "head": 5,
      "deprel": "det",
      "start_char": 14,
      "end_char": 17,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "prior",
      "lemma": "prior",
      "upos": "ADJ",
      "xpos": "JJ",
      "head": 5,
      "deprel": "amod",
      "start_char": 18,
      "end_char": 23,
 